# News Engagement Prediction

Create a system that predicts the engagement of a given news article based on its content (title and/or article text):
* Data from https://webhose.io/free-datasets/popular-news-articles/
* word2vec-CNN-BiLSTM model
* Trained on Triton supercomputer GPUs
* Created in Jupyter Notebook using Pytorch
* Deliverable as requestable API on AWS
* Possibly create a dashboard for user to test and rank different word choices

## Preprocessing

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import json
import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import random
import math

import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as ticker
import matplotlib.dates as md

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from dateutil.relativedelta import relativedelta

import preprocess as pp
import graph as gph
import hparams as hp
import model as ml
import train as tr
import postprocess as pop
import analysis as al

In [20]:
# insert other choices for model
model = ml.MODELS['cnn-bilstm']
hps = hp.setup_hparams(model, dict())

In [3]:
# max_count = -1 will use entire dataset
data_all = pp.read_data()
data_all = pp.remove_keys(data_all)
data_all = pp.cal_all_engagements(data_all)

titles = pp.get_titles(data_all)
data_all_ordered = pp.order_keys(data_all)

In [4]:
pp.pprint(data_all_ordered[0])

{
    "title": "Here are The Funniest Reaction Memes To Beyonce's Pregnancy | Angie Martinez | Power 105.1 FM",
    "sanitized_title": "here are the funniest reaction memes to beyonces pregnancy",
    "text": "Here are The Funniest Reaction Memes To Beyonce's Pregnancy posted by Gabriel Pabon - \nThe internet wastes no time when reacting to big news especially when it comes to Beyonce . \nCheck out the funniest memes from today's announcement below. A photo posted by HONEY GERMAN (@honeygerman) on Feb 1, 2017 at 1:03pm PST A photo posted by HONEY GERMAN (@honeygerman) on Feb 1, 2017 at 12:57pm PST A photo posted by HONEY GERMAN (@honeygerman) on Feb 1, 2017 at 11:40am PST A photo posted by SSquared Podcast/Radio Show (@teamssquared) on Feb 1, 2017 at 2:07pm PST A photo posted by D-Roc \u264f\ufe0f (@inked_scorpio) on Feb 1, 2017 at 2:07pm PST A photo posted by At Random With Yeasha (@random_yeasha) on Feb 1, 2017 at 2:05pm PST Left or Right? \uf62d #beyonce pic.twitter.com/PPhB9QmUAy v

In [9]:
# currently using only titles to save on training time
all_text = pp.get_all_text(titles)
words = pp.get_words(all_text)

# score can be 'original', 'log_weigh', 'log_no_weigh', 'no_log_weigh', 'no_log_no_weigh'
# note: using no_log is a dumb idea
scores = pp.get_scores(data_all_ordered)
mean, std = pp.get_mean_std(scores)
scores = pp.scale_data(scores, mean, std)

In [22]:
tokens = pp.tokenize_words(words)
title_tokens = pp.tokenize_titles(titles, tokens)
title_lengths = pp.get_title_lengths(title_tokens)
title_tokens, titles, scores = pp.remove_shorts(title_tokens, titles, scores, min_len=3)
padded_titles = pp.pad_titles(title_tokens)

hps = pp.update_hps(hps, tokens)

In [23]:
train_x, val_x, test_x = pp.split_data(padded_titles)
train_y, val_y, test_y = pp.split_data(scores)

print("Train Size: {}".format(train_x.shape),
      "\nValidation Size: {}".format(val_x.shape),
      "\nTest Size: {}".format(test_x.shape))

Train Size: (786, 25) 
Validation Size: (98, 25) 
Test Size: (99, 25)


In [24]:
train_data = pp.create_tensor_dataset(train_x, train_y)
val_data = pp.create_tensor_dataset(val_x, val_y)
test_data = pp.create_tensor_dataset(test_x, test_y)

train_loader = pp.create_loader(train_data)
val_loader = pp.create_loader(val_data)
test_loader = pp.create_loader(test_data)

In [26]:
model = ml.CNN_BiLSTM(hps)
print(model)

CNN_BiLSTM(
  (embed): Embedding(4317, 200, padding_idx=0)
  (bilstm): LSTM(200, 200, num_layers=2, dropout=0.1, bidirectional=True)
  (fc1): Linear(in_features=700, out_features=350, bias=True)
  (fc2): Linear(in_features=350, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
opt = tr.get_optimizer()
criterion = tr.get_criterion()